In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

# Create dataset of masked rasters

**Input:** 

Raw data 
* .SAFE JP2 
* .shp  files

**Output:**

Two types of masks:

1. Original data - just the farm, sliced out from the spectral bands
    - `data/interim/masks/{train/test}/{date}/{farm_id}/{band}.npy`
2. Original masks zero-padded to be the same size as the largest farm
    - `data/interim/masks/{train/test}/{date}/{farm_id}/{band}_zp.npy`



## Overview

Very similar to making baseline dataset, except that now the process stops at masking and the masked farms are output as `.npy` files.

In this notebook, we will write a prototype to load the data from one band in one timestamp and create masks for all the farms in the training set.


In [2]:
import os
import rasterio
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from glob import glob

import sys
sys.path.append('../')
from config import interim_data_dir, raw_data_dir

from src.utils import get_img_bands, get_safe_dirs, date_from_safedir, band_from_imgpath, read_shapefile, mask_raster, safe_create_dir

## 1. Original Data - just mask out farms

In [3]:
dataset = 'train'

masks_dir = os.path.join(interim_data_dir, 'masks')
safe_create_dir(masks_dir)

# train / test dir under masks
dataset_dir = os.path.join(masks_dir, dataset)
safe_create_dir(dataset_dir)

shp_df = read_shapefile(dataset)

shp_df.head()

Shell is not a LinearRing
Shell is not a LinearRing
Shell is not a LinearRing
Shell is not a LinearRing
Shell is not a LinearRing
Shell is not a LinearRing
Shell is not a LinearRing
Shell is not a LinearRing
Shell is not a LinearRing
Shell is not a LinearRing
Shell is not a LinearRing
Shell is not a LinearRing
Shell is not a LinearRing
Shell is not a LinearRing
Shell is not a LinearRing
Shell is not a LinearRing
Shell is not a LinearRing
Shell is not a LinearRing
Shell is not a LinearRing
Shell is not a LinearRing
Shell is not a LinearRing
Shell is not a LinearRing
Shell is not a LinearRing
Shell is not a LinearRing
Shell is not a LinearRing
Shell is not a LinearRing
Shell is not a LinearRing
Shell is not a LinearRing
Shell is not a LinearRing
Shell is not a LinearRing
Shell is not a LinearRing
Shell is not a LinearRing
Shell is not a LinearRing
Shell is not a LinearRing
Shell is not a LinearRing
Shell is not a LinearRing
Shell is not a LinearRing
Shell is not a LinearRing
Shell is not

,Area,Subregion,Crop_Id_Ne,geometry
Field_Id,,,,


## Create original masks

In [ ]:
from tqdm import tqdm
import gc

# Get list of .SAFE dumps
safe_dirs = get_safe_dirs()

# Choose the first SAFE file for prototyping
safe_dir = safe_dirs[0]

# Get the date of the data dump
date = date_from_safedir(safe_dir)

out_dir = os.path.join(dataset_dir, date)
safe_create_dir(out_dir)

img_fpaths = get_img_bands(safe_dir)

for img_fpath in tqdm(img_fpaths, desc='bands'):
    
    band = band_from_imgpath(img_fpath)

    with rasterio.open(img_fpath) as raster:
        masks = mask_raster(shp_df.geometry, raster, return_missing=dataset == 'test')

    # Save masks for each farm
    for farm_id in masks.keys():
        mask = masks[farm_id]
        
        farm_dir = os.path.join(out_dir, str(farm_id))
        safe_create_dir(farm_dir)

        mask_fname = os.path.join(farm_dir, f'{band}.npy')

        np.save(mask_fname, mask)

    del masks
    gc.collect()

### Run again for JEP of the same dump

In [ ]:
safe_dir_jfp = safe_dir.replace('JEP', 'JFP')

In [ ]:
# Get the date of the data dump
date = date_from_safedir(safe_dir_jfp)

out_dir = os.path.join(dataset_dir, date)
safe_create_dir(out_dir)

img_fpaths = get_img_bands(safe_dir_jfp)

for img_fpath in tqdm(img_fpaths, desc='bands'):
    
    band = band_from_imgpath(img_fpath)

    with rasterio.open(img_fpath) as raster:
        masks = mask_raster(shp_df.geometry, raster, return_missing=dataset == 'test')

    # Save masks for each farm
    for farm_id in masks.keys():
        mask = masks[farm_id]
        
        farm_dir = os.path.join(out_dir, str(farm_id))
        safe_create_dir(farm_dir)

        mask_fname = os.path.join(farm_dir, f'{band}.npy')

        np.save(mask_fname, mask)

    del masks
    gc.collect()

## Find max width and height for each spectral resolution

There are 3 resolution groups: 10, 20 and 60m. From the documentation we know which bands are what resolution. A smaller (higher) resolution would mean larger images ito pixels. 

So we need to find the maximum height and width that any farm has (in train and testing set) for the 3 resolution groups.

**Output**:

* 60: (max_width, max_height)
* 20: (max_width, max_height)
* 10: (max_width, max_height)

In [ ]:
path = '../data/interim/masks/train/2017-07-10'
farms = glob(path+'/*')

resolution_groups = {
    '60': ['B01', 'B09', 'B10'],
    '20': ['B05', 'B06', 'B07', 'B8A', 'B11', 'B12'],
    '10': ['B02', 'B03', 'B04', 'B08', 'TCI']
}

max_widths = {'60': 0, '20': 0, '10': 0}
max_heights = {'60': 0, '20': 0, '10': 0}
for farm_path in tqdm(farms):
    farm_id = os.path.basename(farm_path)
    files = [f for f in glob(farm_path+'/*.npy')]
    
    for file in files:
        band = os.path.basename(file).split('.')[0]
        
        arr = np.load(file)
        
        width, height = arr.shape
        
        res_group = [grp for grp, bands in resolution_groups.items() if band in bands][0]
        
        if width > max_widths[res_group]:
            max_widths[res_group] = width
        
        if height > max_heights[res_group]:
            max_heights[res_group] = height

### Combine the maximum dimensions into one dict

In [ ]:
from pprint import pprint
max_dims = {}
for grp in ['60','20','10']:
    max_dims[grp] = (max_widths[grp], max_heights[grp])
pprint(max_dims)

#### Create squares from maximum dims

In [ ]:
MAX_DIMS = {
    '10': (90,90),
    '20': (50,50),
    '60': (20,20)
}

In [ ]:
del widths
del heights

## 2. Create zero-padded masks

### Test on two images first - one large, one random

In [ ]:
import cv2

def zeropad_img(img, shape=MAX_DIMS):

    # Size of border
    v_border = int(np.ceil((shape[0] - img.shape[0])/2))
    h_border = int(np.ceil((shape[1] - img.shape[1])/2))
    
    v_diff = shape[0] - (img.shape[0] + 2*v_border)
    h_diff = shape[1] - (img.shape[1] + 2*h_border)
    
    new_img = cv2.copyMakeBorder(
        img, 
        top=v_border, bottom=v_border+v_diff, 
        left=h_border, right=h_border+h_diff,
        borderType=cv2.BORDER_CONSTANT, value=0
    )
    
    assert new_img.shape == shape, 'zero padding issue'
    
    return new_img

def get_res_group(band):
    """
    Get the resolution group of a band
    """
    resolution_groups = {
        '60': ['B01', 'B09', 'B10'],
        '20': ['B05', 'B06', 'B07', 'B8A', 'B11', 'B12'],
        '10': ['B02', 'B03', 'B04', 'B08', 'TCI']
    }
    [res_group] = [grp for grp, bands in resolution_groups.items() if band in bands]
    
    return res_group


img_fpath = '/Users/renier.botha/dev/personal/ds/zindi/farm-pin/data/raw/S2A_MSIL1C_20170710T082011_N0205_R121_T34JFP_20170710T084244.SAFE/GRANULE/L1C_T34JFP_A010700_20170710T084244/IMG_DATA/T34JFP_20170710T082011_B02.jp2'

# Get the resolution group of img
band = band_from_imgpath(img_fpath)
res_group = get_res_group(band)

with rasterio.open(img_fpath) as raster:
    masks = mask_raster(shp_df.geometry, raster, return_missing=dataset == 'test')
    

# Select a large farm
large_farm = [mask for mask in masks.values() if mask.shape[0] > 80][0]

# Select a random farm
np.random.seed(420)
# Pick a random farm to process
random_farm = masks[np.random.choice(list(masks.keys()))]


# Zero pad the both farms
large_farm_zp = zeropad_img(large_farm, shape=MAX_DIMS[res_group])
random_farm_zp = zeropad_img(random_farm, shape=MAX_DIMS[res_group])

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(15,10))
ax=axes[0][0]
ax.imshow(np.log10(large_farm), cmap='gray')
ax.set_title('Large farm')

# Zero pad the large farm
ax=axes[0][1]
ax.imshow(np.log10(large_farm_zp), cmap='gray')
ax.set_title('Zero-padded large farm')


ax=axes[1][0]
ax.imshow(np.log10(mask), cmap='gray')
ax.set_title('Randomly selected farm')

ax = axes[1][1]
ax.imshow(np.log10(random_farm_zp), cmap='gray')
ax.set_title('Zero-padded farm')
plt.show()

## Repeat zero pad for all images in one band

In [ ]:
img_fpaths = get_img_bands(safe_dir)

for img_fpath in tqdm(img_fpaths, desc='bands'):
    
    band = band_from_imgpath(img_fpath)
    
    res_group = get_res_group(band)

    with rasterio.open(img_fpath) as raster:
        masks = mask_raster(shp_df.geometry, raster, return_missing=dataset == 'test')
        
        # Create zeropadded masks
        zp_masks = {id: zeropad_img(mask, shape=MAX_DIMS[res_group]) for id, mask in masks.items()}

    # Save zero-padded masks for each farm
    for farm_id, zp_mask in zp_masks.items():
        
        farm_dir = os.path.join(out_dir, str(farm_id))
        safe_create_dir(farm_dir)

        mask_fname = os.path.join(farm_dir, f'{band}_zp.npy')

        np.save(mask_fname, zp_mask)

    del masks
    del zp_masks
    gc.collect()

## Check dirs by date

Check that each date has two directories (JEP and JFP)

In [ ]:
dirs_by_date={}
for dir in safe_dirs:
    date = date_from_safedir(dir)
    if date not in dirs_by_date:
        dirs_by_date[date] = [dir]
    else:
        dirs_by_date[date].append(dir)
        
dirs_by_date